## Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

In [1]:
! pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import cv2
import os

In [12]:
cascPathface = cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)

In [18]:
cartoon_img_path = r"C:\Users\veeraj\OneDrive\Desktop\Shrey\tom.png"
cartoon_img = cv2.imread(cartoon_img_path, cv2.IMREAD_UNCHANGED)

In [ ]:
if cartoon_img is None:
    raise FileNotFoundError(f"Cartoon image not found at path: {cartoon_img_path}")


video_capture = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(r'C:\Users\veeraj\OneDrive\Desktop\Shrey\shrey.avi', fourcc, 20.0, (640, 480))

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x, y, w, h) in faces:
        cartoon_resized = cv2.resize(cartoon_img, (w, h))
        
        if cartoon_resized.shape[2] == 4:
            b, g, r, a = cv2.split(cartoon_resized)
            mask = a
            mask_inv = cv2.bitwise_not(mask)
            roi = frame[y:y+h, x:x+w]
            mask = cv2.resize(mask, (w, h))
            mask_inv = cv2.resize(mask_inv, (w, h))
            
            # Apply mask
            cartoon_fg = cv2.bitwise_and(cartoon_resized[:, :, :3], cartoon_resized[:, :, :3], mask=mask)
            roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
            result = cv2.add(cartoon_fg, roi_bg)
            
        else:
            result = cartoon_resized
        
        frame[y:y+h, x:x+w] = result
    

    out.write(frame)
    cv2.imshow('Cartoonified Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
out.release()
cv2.destroyAllWindows()